<div class="alert alert-success" role="quote" style="text-align:center;font-size:120%;;">
  In the name of Allah, the most Merciful and the most Beneficent!</div>

In [ ]:
import numpy as np
import pandas as pd
import jpx_tokyo_market_prediction
from lightgbm import LGBMRegressor
import optuna.integration.lightgbm as lgb
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

In [ ]:
df = pd.read_csv("../input/jpx-tokyo-stock-exchange-prediction/supplemental_files/stock_prices.csv")

In [ ]:
df.info()

In [ ]:
#NDAYS = 34
#lastdays = prices[prices["Date"]>=prices.Date.iat[-2000*NDAYS]].reset_index(drop=True)

In [ ]:
#lastdays = pd.DataFrame(prices.groupby("SecuritiesCode").Target.mean())
def get_avg(_id_):
    return lastdays.loc[_id_]
#prices["Avg"] = prices["SecuritiesCode"].apply(get_avg)

In [ ]:
df.Date.head(1)

In [ ]:
df.Date = pd.to_datetime(df.Date)
#prices['Date'] = prices['Date'].dt.strftime("%Y%m%d").astype(int)

In [ ]:
X = df.copy()
Y = X.pop('Target')

# First for checking drop some columns from X

In [ ]:
real_X = X.iloc[:, 1:7]


In [ ]:
def date_to_classic(df):
    dt = df.Date
    df['Year'] = dt.dt.year
    df['Month'] = dt.dt.month
    df['Week'] = dt.dt.week
    df['quarter'] = dt.dt.quarter
    df.drop("Date", axis=1, inplace=True)
    return df

In [ ]:
date_to_classic(real_X)

# !Generated Features!

In [ ]:
def feature_generator(df):
    #moving average
    
    #exponential moving average
    #disparity
    #momentum1
    #momentum2
    #rate f change
    #Relative Strength Index
    pass

In [ ]:
import optuna

def objectives(trial):
    # optunaでのハイパーパラメータサーチ範囲の設定
    params = {
            'num_leaves': trial.suggest_int('num_leaves', 200, 3000),
            'n_estimators': trial.suggest_int('n_estimators', 100, 3000),
            'max_bin': trial.suggest_int('max_bin', 2, 100),
            'learning_rate': trial.suggest_uniform('learning_rate',0, 1),
    }

    model = LGBMRegressor()
    model.fit(real_X,Y)
    score = model.score(real_X,Y)
    return score

In [ ]:
opt = optuna.create_study(direction='maximize',sampler=optuna.samplers.RandomSampler(seed=0))
opt.optimize(objectives, n_trials=2) # for now lets make it 2 insted of 50

# 最適パラメータ取得
trial = opt.best_trial
params_best = dict(trial.params.items())
params_best['random_seed'] = 16
    
# 最適パラメータで学習/予測    
model_o = LGBMRegressor() #**params, i skip it

In [ ]:
model_o.fit(real_X,Y)

In [ ]:
env = jpx_tokyo_market_prediction.make_env()

In [ ]:
iter_test = env.iter_test()

In [ ]:
for (prices, options, financials, trades, secondary_prices, sample_prediction) in iter_test:
    ds=[prices, options, financials, trades, secondary_prices, sample_prediction]
    df = sample_prediction[["Date","SecuritiesCode"]]
    df["High"] = prices["High"]
    df["Open"] = prices["Open"]
    df["Close"] = prices["Close"]
    df["Low"] = prices["Low"]
    df.Date = pd.to_datetime(df.Date)
    df = date_to_classic(df)
    print(df)
    sample_prediction["Prediction"] = model_o.predict(df)
    sample_prediction = sample_prediction.sort_values(by = "Prediction", ascending=False)
    sample_prediction.Rank = np.arange(0,2000)
    sample_prediction = sample_prediction.sort_values(by = "SecuritiesCode", ascending=True)
    sample_prediction.drop(["Prediction"],axis=1)
    submission = sample_prediction[["Date","SecuritiesCode","Rank"]]
    env.predict(submission)
    #sample_prediction["Prediction"] = model_o.predict(ndf)
    #sample_prediction = sample_prediction.sort_values(by = "Prediction", ascending=False)
    #sample_prediction.Rank = np.arange(0,2000)
    #sample_prediction = sample_prediction.sort_values(by = "SecuritiesCode", ascending=True)
    #sample_prediction.drop(["Prediction"],axis=1)
    #submission = sample_prediction[["Date","SecuritiesCode","Rank"]]
    #env.predict(submission)